# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y\\

In [7]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
from nltk import word_tokenize, WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_recall_fscore_support,classification_report
from sklearn.svm import SVC, LinearSVC

import re
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet');

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [8]:
# load data from database
engine = create_engine('sqlite:///Database.db')
df = pd.read_sql_table('Table', engine, index_col='id')

In [9]:
df.columns

Index(['message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [10]:
X = df['message']
y = df.drop(['message', 'original','genre'],axis=1)

### 2. Write a tokenization function to process your text data

In [5]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]"," ", text)
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    tokens = [stemmer.stem(lemmatizer.lemmatize(token)) for token in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
TFIDF = TfidfVectorizer(stop_words=None, tokenizer = tokenize, lowercase=True)
multioutput = MultiOutputClassifier(LogisticRegression(), n_jobs=-1)
pipeline = make_pipeline(TFIDF, multioutput)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...one, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
yhat_test = pipeline.predict(X_test)

In [9]:
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, yhat_test,average='micro')
print('Precision: {}\nRecall: {}\nFscore: {}'.format(precision,recall,fscore))

Precision: 0.8185907046476761
Recall: 0.5450915141430949
Fscore: 0.6544147023571714


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'clf__estimator__C':[0.01,0.1,0.2,0.5,1,2]}

clf = MultiOutputClassifier(LogisticRegression(), n_jobs=-1)

pipeline = Pipeline([
    ('tfidf', TFIDF),
    ('clf', clf)])

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', refit=True)

cv.fit(X_train, y_train)
yhat_test_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, yhat_test_cv,average='micro')
print('Precision: {}\nRecall: {}\nFscore: {}'.format(precision,recall,fscore))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
parameters = {'clf__estimator__C':[0.25,0.5,1]}

clf = MultiOutputClassifier(LinearSVC(), n_jobs=-1)

pipeline = Pipeline([
    ('tfidf', TFIDF),
    ('clf', clf)])

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', refit=True)

cv.fit(X_train, y_train)
yhat_test_cv = cv.predict(X_test)

In [ ]:
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, yhat_test_cv,average='micro')
print('Precision: {}\nRecall: {}\nFscore: {}'.format(precision,recall,fscore))

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv.best_estimator_,open('best_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.